In [36]:
import graphlab as gl
import numpy as np
from scipy.sparse import *
from scipy import *

In [56]:
data = gl.SFrame.read_csv("/Volumes/EXTRADRIVE/data/netflix_sample.csv")

PROGRESS: Finished parsing file /Volumes/EXTRADRIVE/data/netflix_sample.csv
PROGRESS: Parsing completed. Parsed 100 lines in 1.71772 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Read 2329258 lines. Lines per second: 1.05647e+06
PROGRESS: Finished parsing file /Volumes/EXTRADRIVE/data/netflix_sample.csv
PROGRESS: Parsing completed. Parsed 3749377 lines in 2.43001 secs.


In [57]:
data

X1,UID,Rating,MID
0,369848,2,5279
1,2623758,2,5279
2,2293400,1,5279
3,1542310,3,5279
4,1216170,2,5279
5,2326917,2,5279
6,1221134,2,5279
7,1357230,2,5279
8,1602955,2,5279
9,2544102,5,5279


In [101]:
numUsers = len(data["UID"].unique())
numUsers

17770

In [102]:
numMovies = len(data["MID"].unique())
numMovies

17695

In [60]:
users = data["UID"].unique()

In [35]:
movies = data["MID"].unique()

In [85]:
usersDict = gl.SArray([{ID: idx for idx, ID in enumerate(users)}])
moviesDict = gl.SArray([{ID: idx for idx, ID in enumerate(movies)}])

In [98]:
data["rowIdx"] = data.apply(lambda x: usersDict[0].get(x["UID"]))
data["columnIdx"] = data.apply(lambda x: moviesDict[0].get(x["MID"]))

In [99]:
data

X1,UID,Rating,MID,rowIdx,columnIdx
0,369848,2,5279,13016,8671
1,2623758,2,5279,13309,8671
2,2293400,1,5279,10184,8671
3,1542310,3,5279,2949,8671
4,1216170,2,5279,9047,8671
5,2326917,2,5279,13113,8671
6,1221134,2,5279,4833,8671
7,1357230,2,5279,5467,8671
8,1602955,2,5279,6217,8671
9,2544102,5,5279,6101,8671


In [103]:
row = data["rowIdx"]
col = data["columnIdx"]
ratings = data["Rating"]
UM = csr_matrix( (ratings,(row,col)), shape=(numUsers,numMovies) )

In [104]:
UM.get_shape()

(17770, 17695)

In [113]:
UM[2949, 8671]

3

In [114]:
UM_sum = UM.sum()
UM_sum

13505909

In [126]:
UM_mean = UM_sum / float(UM.nnz)
UM_mean

3.6021741745361964

In [136]:
UM_std = sum((UM.toarray() - UM_mean)**2) / float(numMovies*numUsers)
UM_std

12.835028109760003

In [137]:
UM_norm = (UM.toarray() - UM_mean) / UM_std
UM_norm

array([[-0.28065183, -0.28065183, -0.28065183, ..., -0.28065183,
        -0.28065183, -0.28065183],
       [-0.28065183, -0.28065183, -0.28065183, ..., -0.28065183,
        -0.28065183, -0.28065183],
       [-0.28065183, -0.28065183, -0.28065183, ..., -0.28065183,
        -0.28065183, -0.28065183],
       ..., 
       [-0.28065183, -0.28065183, -0.28065183, ..., -0.28065183,
        -0.28065183, -0.28065183],
       [-0.28065183, -0.28065183, -0.28065183, ..., -0.28065183,
        -0.28065183, -0.28065183],
       [-0.28065183, -0.28065183, -0.28065183, ..., -0.28065183,
        -0.28065183, -0.28065183]])

In [138]:
UM_norm = csr_matrix(UM_norm)